In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.kernel_ridge import KernelRidge
from numpy.linalg import svd
from scipy import signal, stats
from scipy.stats import t
import h5py
import PIL
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def pearson_correlation_coefficient(x: np.ndarray, y: np.ndarray, axis: int) -> np.ndarray:
    r = (np.nan_to_num(stats.zscore(x)) * np.nan_to_num(stats.zscore(y))).mean(axis)
    p = 2 * t.sf(np.abs(r / np.sqrt((1 - r ** 2) / (x.shape[0] - 2))), x.shape[0] - 2)
    return r, p

In [ ]:
class KernelRidgeCV:
    def __init__(self, kernel, target, n_lambdas):
        self.kernel = kernel
        self.target = target
        self.n_lambdas = n_lambdas
        self._lambdas = None
        self._df = None

    @property
    def lambdas(self):
        if self._lambdas is not None:   
            return self._lambdas

        # singular value decomposition
        s = svd(self.kernel)[1]
        s = s[s > 0]
        
        self._lambdas = np.full((self.n_lambdas), np.nan)
        length = s.shape[0]
        self._df = np.linspace(length, 1, self.n_lambdas)
        mean = np.mean(1/s)
        f = lambda df, lamb: df - np.sum(s / (s + lamb))
        f_prime = lambda lamb: np.sum(s / (s + lamb)**2)

        # get all the lambdas
        for i in range(1, self.n_lambdas):
            if i == 1:
                self._lambdas[i] = 0
            else:
                self._lambdas[i] = self._lambdas[i-1]
            self._lambdas[i] = max(self._lambdas[i], (length / self._df[i] - 1) / mean)
            temp = f(self._df[i], self._lambdas[i])
            while abs(temp) > 1e-10:
                self._lambdas[i] = max(0, self._lambdas[i] - temp / f_prime(self._lambdas[i]))
                temp = f(self._df[i], self._lambdas[i])
        return self._lambdas[1:]


    def train(self, X):
        best_model, best_error = None, np.inf

        # CV over all the lambdas
        for lambda_, df_ in zip(self.lambdas, self._df):
            
            # k-Ridge, where regularization?
            # precomputed, then treat X as kernel (otherwise as X)
            kernel_ridge = KernelRidge(alpha=lambda_)
            kernel_ridge.fit(X, self.target)
            y = kernel_ridge.predict(X)
            error = np.sum(((self.target - y) / (1 - df_ / self.kernel.shape[0])) ** 2)
            if error < best_error:
                best_error = error
                best_model = kernel_ridge
        print("Best error:", best_error, "Alpha: ", best_model.alpha)
        return best_model

# StyleGAN3 (faces)

In [ ]:
f1 = h5py.File("/content/drive/My Drive/faces/GANs_StyleGAN3_normMUA.mat", "r")
x_te = np.delete(np.array(f1["test_MUA"]), np.arange(320, 384), axis=1)
x_tr = np.delete(np.array(f1["train_MUA"]), np.arange(320, 384), axis=1)
dims = [0, 448, 704, 960]

In [ ]:
# vgg16 for face recognition
ys1 = np.zeros((2, 200, 960))
for i, layer in enumerate(range([0, 2])):
    a_te = np.load("/content/drive/My Drive/faces/vggface_te_%i.npy" % layer).reshape(200, -1)
    a_tr = np.load("/content/drive/My Drive/faces/vggface_tr_%i.npy" % layer).reshape(4000, -1)

    # n x n kernel matrix of pairwise similarity comparisons
    kernel = a_tr @ a_tr.T
    kernel = kernel.astype(float)
    for roi in range(3):
        ridge_cv = KernelRidgeCV(kernel, x_tr[:, dims[roi]:dims[roi+1]], 5)
        model = ridge_cv.train(a_tr)
        ys1[i, :, dims[roi]:dims[roi+1]] = model.predict(a_te)
    np.save("/content/drive/My Drive/faces/ys_vgg16.npy", ys1)

In [ ]:
# z-latents
z_te = np.load("/content/drive/My Drive/faces/z_te.npy")
z_tr = np.load("/content/drive/My Drive/faces/z_tr.npy")
kernel = z_tr @ z_tr.T
kernel = kernel.astype(float)
ys2 = np.zeros((100, 960))
for roi in range(3):
    ridge_cv = KernelRidgeCV(kernel, x_tr[:, dims[roi]:dims[roi+1]], 5)
    model = ridge_cv.train(z_tr)
    ys2[:, dims[roi]:dims[roi+1]] = model.predict(z_te)
np.save("/content/drive/My Drive/faces/ys_z.npy", ys2)

In [ ]:
# w-latents
w_te = np.load("/content/drive/My Drive/faces/w_te.npy")[:, 0]
w_tr = np.load("/content/drive/My Drive/faces/w_tr.npy")[:, 0]
kernel = w_tr @ w_tr.T
kernel = kernel.astype(float)
ys3 = np.zeros((100, 960))
for roi in range(3):
    ridge_cv = KernelRidgeCV(kernel, x_tr[:, dims[roi]:dims[roi+1]], 5)
    model = ridge_cv.train(w_tr)
    ys3[:, dims[roi]:dims[roi+1]] = model.predict(w_te)
np.save("/content/drive/My Drive/faces/ys_w.npy", ys3)

In [ ]:
# CLIP latents
clip_te = np.load("/content/drive/My Drive/faces/clip_te.npy")
clip_tr = np.load("/content/drive/My Drive/faces/clip_tr.npy")
kernel = clip_tr @ clip_tr.T
kernel = kernel.astype(float)
ys4 = np.zeros((100, 960))
for roi in range(3):
    ridge_cv = KernelRidgeCV(kernel, x_tr[:, dims[roi]:dims[roi+1]], 5)
    model = ridge_cv.train(clip_tr)
    ys4[:, dims[roi]:dims[roi+1]] = model.predict(clip_te)
np.save("/content/drive/My Drive/faces/ys_clip.npy", ys4)

# StyleGAN-XL (natural images)

In [ ]:
f1 = h5py.File("/content/drive/My Drive/images/GANs_StyleGAN_XL_normMUA.mat", "r")
dims = [0, 448, 704, 960]

# delete broken microelectrode array
x_te = np.delete(np.array(f1["test_MUA"]), np.arange(320, 384), axis=1)
x_tr = np.delete(np.array(f1["train_MUA"]), np.arange(320, 384), axis=1)

In [ ]:
# vgg16 for object recognition
ys1 = np.zeros((2, 200, 960))
for i, layer in enumerate(range([0, 2])):
    a_te = np.load("/content/drive/My Drive/images/vgg_te_%i.npy" % layer).reshape(200, -1)
    a_tr = np.load("/content/drive/My Drive/images/vgg_tr_%i.npy" % layer).reshape(4000, -1)
    kernel = a_tr @ a_tr.T
    kernel = kernel.astype(float)
    for roi in range(3):
        ridge_cv = KernelRidgeCV(kernel, x_tr[:, dims[roi]:dims[roi+1]], 5)
        model = ridge_cv.train(a_tr)
        ys1[i, :, dims[roi]:dims[roi+1]] = model.predict(a_te)
np.save("/content/drive/My Drive/images/ys_vgg16.npy", ys1)

In [ ]:
# z-latents
z_te = np.load("/content/drive/My Drive/images/z_te.npy")
z_tr = np.load("/content/drive/My Drive/images/z_tr.npy")
kernel = z_tr @ z_tr.T
kernel = kernel.astype(float)
ys2 = np.zeros((200, 960))
for roi in range(3):
    ridge_cv = KernelRidgeCV(kernel, x_tr[:, dims[roi]:dims[roi+1]], 5)
    model = ridge_cv.train(z_tr)
    ys2[:, dims[roi]:dims[roi+1]] = model.predict(z_te)
np.save("/content/drive/My Drive/images/ys_z.npy", ys2)

In [ ]:
# w-latents
w_te = np.load("/content/drive/My Drive/images/w_te.npy")[:, 0]
w_tr = np.load("/content/drive/My Drive/images/w_tr.npy")[:, 0]
kernel = w_tr @ w_tr.T
kernel = kernel.astype(float)
ys3 = np.zeros((200, 960))
for roi in range(3):
    ridge_cv = KernelRidgeCV(kernel, x_tr[:, dims[roi]:dims[roi+1]], 5)
    model = ridge_cv.train(w_tr)
    ys3[:, dims[roi]:dims[roi+1]] = model.predict(w_te)
np.save("/content/drive/My Drive/images/ys_w.npy", ys3)

In [ ]:
# CLIP latents
clip_te = np.load("/content/drive/My Drive/images/clip_te.npy")
clip_tr = np.load("/content/drive/My Drive/images/clip_tr.npy")
kernel = clip_tr @ clip_tr.T
kernel = kernel.astype(float)
ys4 = np.zeros((200, 960))
for roi in range(3):
    ridge_cv = KernelRidgeCV(kernel, x_tr[:, dims[roi]:dims[roi+1]], 5)
    model = ridge_cv.train(clip_tr)
    ys4[:, dims[roi]:dims[roi+1]] = model.predict(clip_te)
np.save("/content/drive/My Drive/images/ys_clip.npy", ys4)